In [1]:
import numpy as np
import pandas as pd
import os
import glob

In [2]:
os.chdir('E:\gr')
path ="E:\gr"
filenames = glob.glob(path + "/*.csv")
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename,encoding="gbk",skiprows=7))
gr = pd.concat(dfs, ignore_index=True)

In [3]:
gr=gr.dropna()

In [4]:
gr=gr[~gr['开始时间'].str.contains('开始时间')]

In [5]:
a=gr['开始时间'][0][6:10]+"_"+gr['开始时间'][0][0:2]+"_"+gr['开始时间'][0][3:5]

In [6]:
gr.to_excel(f'{a}_干扰明细.xlsx',index=False)

In [7]:
gr['系统上行每个PRB上检测到的干扰噪声的平均值 (毫瓦分贝)']=gr['系统上行每个PRB上检测到的干扰噪声的平均值 (毫瓦分贝)'].replace('NIL',-120)
gr['100个PRB干扰的平均值 (毫瓦分贝)']=gr['100个PRB干扰的平均值 (毫瓦分贝)'].replace('NIL',-120)

In [8]:
gr[['系统上行每个PRB上检测到的干扰噪声的平均值 (毫瓦分贝)','100个PRB干扰的平均值 (毫瓦分贝)']]=gr[['系统上行每个PRB上检测到的干扰噪声的平均值 (毫瓦分贝)','100个PRB干扰的平均值 (毫瓦分贝)']].astype(float)

In [9]:
gr['小区']=gr['小区'].astype(str)

In [10]:
def names(a1):
     return(a1.split('小区名称=')[1].split(',')[0])

In [11]:
gr['小区']=gr['小区'].map(names)

In [12]:
gr['区县']=gr['网元名称'].str[2:4]

In [13]:
def gr1(a):
    if a>float(-100):
        return 1
    else:
        return 0
def gr2(b):
    if b>float(-105):
        return 1
    else:
        return 0
def gr3(c):
    if c>float(-110):
        return 1
    else:
        return 0

In [14]:
gr['100个PRB干扰的平均值是否大于-100']=gr.apply(lambda x: gr1(x['100个PRB干扰的平均值 (毫瓦分贝)']),axis=1)
gr['100个PRB干扰的平均值是否大于-105']=gr.apply(lambda y: gr2(y['100个PRB干扰的平均值 (毫瓦分贝)']),axis=1)
gr['100个PRB干扰的平均值是否大于-110']=gr.apply(lambda z: gr3(z['100个PRB干扰的平均值 (毫瓦分贝)']),axis=1)

In [15]:
gr=pd.pivot_table(gr,values=['100个PRB干扰的平均值是否大于-100','100个PRB干扰的平均值是否大于-105','100个PRB干扰的平均值是否大于-110'],index=['区县','开始时间'],aggfunc={'100个PRB干扰的平均值是否大于-100':np.sum,'100个PRB干扰的平均值是否大于-105':np.sum,'100个PRB干扰的平均值是否大于-110':np.sum},margins=True,margins_name='总计')

In [16]:
gr=gr.reset_index('开始时间')

In [17]:
gr=gr.loc[[ '伊川', '偃师',  '孟津', '宜阳', '嵩县',  '新区','吉利' ,'新安',
       '栾川',  '汝阳','洛宁',  '涧西',  '西工','总计']]

In [18]:
gr=gr.reset_index()

In [19]:
gr.rename(columns={'100个PRB干扰的平均值是否大于-100':'大于-100干扰小区数','100个PRB干扰的平均值是否大于-105':'大于-105干扰小区数','100个PRB干扰的平均值是否大于-110':'大于-110干扰小区数'},inplace=True)

In [20]:
gr=gr[['开始时间','区县', '大于-100干扰小区数', '大于-105干扰小区数', '大于-110干扰小区数']]

In [21]:
a=gr['开始时间'][0][6:10]+"_"+gr['开始时间'][0][0:2]+"_"+gr['开始时间'][0][3:5]

In [22]:
gr.to_excel(f'{a}_干扰统计.xlsx',index=False)

In [23]:
gr

,开始时间,区县,大于-100干扰小区数,大于-105干扰小区数,大于-110干扰小区数
0,10/06/2020 09:00:00,伊川,1,2,32
1,10/06/2020 09:00:00,偃师,0,3,48
2,10/06/2020 09:00:00,孟津,0,2,18
3,10/06/2020 09:00:00,宜阳,2,3,21
4,10/06/2020 09:00:00,嵩县,0,0,14
5,10/06/2020 09:00:00,新区,0,1,3
6,10/06/2020 09:00:00,吉利,0,0,3
7,10/06/2020 09:00:00,新安,0,1,19
8,10/06/2020 09:00:00,栾川,0,1,12
9,10/06/2020 09:00:00,汝阳,0,4,27
